# Stage 0

We start with the [initial puzzle](https://www.gsmg.io/puzzle)

![this image](./puzzle.png)

The first hint given was something along the lines of "follow the white rabbit".

We see a 14x14 matrix of squares of 4 different colours. The most important thing to recognize at this point, and the key so solving it, is that the coloured squares are always 8 sqaures apart when going counter clockwise starting at the top left. Our next leap of logic is that there are 8 bits to a byte, and we can presume that white=0 and black=1. For the yellow and blue, we can discover this programmatically. For now let's list off the colors to make it easier to solve:
```
00110b0010110y
11b1001110b011
1101110b001001
0110b000011101
0b1000110y0110
100110y010y011
100b1100010y00
b11000000010y0
00011b0111110b
11b111y0110001
1101000y011011
11110010b01100
0b0111010y0110
01b0110110b011
```


In [1]:
lines='''00110b0010110y
11b1001110b011
1101110b001001
0110b000011101
0b1000110y0110
100110y010y011
100b1100010y00
b11000000010y0
00011b0111110b
11b111y0110001
1101000y011011
11110010b01100
0b0111010y0110
01b0110110b011'''.split('\n')

def top(arr):
    ''' return the top '''
    return reversed(arr[0]), arr[1:]

def left(arr):
    ''' return the first element of each line'''
    new_lines = []
    out = []
    for line in arr:
        out.append(line[0])
        new_lines.append(line[1:])
    return ''.join(out), new_lines

def bottom(arr):
    '''return the bottom line'''
    return arr[-1], arr[:-1]

def right(arr):
    '''return the last element of each line'''
    new_lines=[]
    out=[]
    for line in arr:
        out.append(line[-1])
        new_lines.append(line[:-1])
    return ''.join(reversed(out)), new_lines

ordering = [left, bottom, right, top]
out =[]
while len(lines):
    for fn in ordering:
        tout,lines = fn(lines)
        out.extend(tout)
out_string=''.join(out)
byte_strings =[out_string[i:i+8] for i in range(0, len(out_string), 8)]
print(byte_strings)

['0110011b', '0111001b', '0110110b', '0110011b', '0010111y', '0110100b', '0110111b', '0010111b', '0111010y', '0110100y', '0110010b', '0111001b', '0110010b', '0110010b', '0110010y', '0110100b', '0111001b', '0111000y', '0110110y', '0110000b', '0110111y', '0111010y', '0110010b', '0110010y', '0000']


Now we have unravelled the binary code, we can figure out what they correspond to. The next thing to notice is that none of the bytes start with a "1", this is very characteristic of ascii. If we assume that they're binary representations of ascii, we can check both possibilities of yellow/blue to see if we can decide which is 1 and white is 0.

In [2]:
import string
# the binary representation of all the printable ascii
ascii_bytes = {c: bin(ord(c))[2:].rjust(8,'0') for c in string.printable}

# for each unknown byte, what printable ascii has a prefix match?
for unknown_byte in byte_strings:
    possible = [(c,binrep) for c,binrep in ascii_bytes.items() if unknown_byte.startswith(binrep[:7])]
    print(f"{unknown_byte}: {possible}")


0110011b: [('f', '01100110'), ('g', '01100111')]
0111001b: [('r', '01110010'), ('s', '01110011')]
0110110b: [('l', '01101100'), ('m', '01101101')]
0110011b: [('f', '01100110'), ('g', '01100111')]
0010111y: [('.', '00101110'), ('/', '00101111')]
0110100b: [('h', '01101000'), ('i', '01101001')]
0110111b: [('n', '01101110'), ('o', '01101111')]
0010111b: [('.', '00101110'), ('/', '00101111')]
0111010y: [('t', '01110100'), ('u', '01110101')]
0110100y: [('h', '01101000'), ('i', '01101001')]
0110010b: [('d', '01100100'), ('e', '01100101')]
0111001b: [('r', '01110010'), ('s', '01110011')]
0110010b: [('d', '01100100'), ('e', '01100101')]
0110010b: [('d', '01100100'), ('e', '01100101')]
0110010y: [('d', '01100100'), ('e', '01100101')]
0110100b: [('h', '01101000'), ('i', '01101001')]
0111001b: [('r', '01110010'), ('s', '01110011')]
0111000y: [('p', '01110000'), ('q', '01110001')]
0110110y: [('l', '01101100'), ('m', '01101101')]
0110000b: [('a', '01100001'), ('`', '01100000')]
0110111y: [('n', '01

Ok, so that isn't quite enough information to easily determine the values of yellow, and blue, however we do see that if b == 1 the first few bytes are "gsmg" which is the url we found the image at, and looking at the next few bytes, we can see if y==0, then we get "gsmg.io" the url of the puzzle image. 
Let's try it:

In [3]:
import binascii

def translate(s):
    return s.replace('b','1').replace('y','0')

def to_ascii(binary_string):
    return int(binary_string, 2).to_bytes(1, 'little').decode()

translated = map(translate, byte_strings)
characters = map(to_ascii, translated)

print(''.join(characters))

gsmg.io/theseedisplanted 


And now we have the url to [phase1](http://gsmg.io/theseedisplanted)

Go to [Walkthrough for Phase 1](/phase1.ipynb)